In [1]:
# Import the needed packages
import numpy as np
import pandas as pd
import bokeh.plotting as bk
import bokeh.io as bi
from bokeh.io import export_svgs
#bi.output_notebook()

# TUBS Rot, Gelb, Orange, Hellgrün, Grün, Dunkelgrün, Hellblau, Blau, Dunkelblau, Violett
TUBScolorscale=[(190,30,60,0.9),
               (255,200,42,0.9),
               (225,109,0,0.9),
               (172,193,58,0.9),
               (109,131,0,0.9),
               (0,83,74,0.9),
               (102,180,211,0.9),
               (0,112,155,0.9),
               (0,63,87,0.9),
               (138,48,127,0.9)]

In [2]:
# Open the data file
# Read the Data
Data = pd.read_csv('./Data/H05_TICONST.csv', sep=';')
# Define the studyname
studyname = "H01_TICONST"
# Give the Columns
Data.columns

Index([u'Unnamed: 0', u'Degree', u'MS_RGA', u'w_MS_RGA', u'Grad_RGA', u'MS_A',
       u'w_MS_A', u'Grad_A', u'MS_D', u'w_MS_D', u'Grad_D', u'w_0.01_RGA',
       u'w_0.01_A', u'w_0.01_D', u'w_0.02_RGA', u'w_0.02_A', u'w_0.02_D',
       u'w_0.1_RGA', u'w_0.1_A', u'w_0.1_D', u'w_0.00666666666667_RGA',
       u'w_0.00666666666667_A', u'w_0.00666666666667_D', u'w_0.005_RGA',
       u'w_0.005_A', u'w_0.005_D'],
      dtype='object')

In [3]:
# Individual Sample Comparision
from bokeh.models import BoxAnnotation, Label

# Get the RGA, Aström and R2D2
sample_RGA = Data[(0.0<Data['MS_RGA']) & (1.2>Data['MS_RGA'])]['MS_RGA']
sample_A = Data[(0.0<Data['MS_RGA']) & (1.2>Data['MS_RGA'])]['MS_A']
sample_D = Data[(0.0<Data['MS_RGA']) & (1.2>Data['MS_RGA'])]['MS_D']

# Show the robustness of Aström relative to RGA
rel_rob_A = sample_A > sample_RGA 
rel_rob_A = rel_rob_A.astype(int)
# Make zeros to 1 and 1 to -1
rel_rob_A[rel_rob_A == 1] = -1
rel_rob_A[rel_rob_A == 0] = 1
# Make the relative gain in robustness
rel_mag_A = np.abs(1-sample_A / sample_RGA)
A = rel_mag_A*rel_rob_A

# Show the robustness of R2 relative to RGA
rel_rob_D = sample_D > sample_RGA 
rel_rob_D = rel_rob_D.astype(int)
# Make zeros to 1 and 1 to -1
rel_rob_D[rel_rob_D == 1] = -1
rel_rob_D[rel_rob_D == 0] = 1
# Make the relative gain in robustness
rel_mag_D = np.abs(1-sample_D / sample_RGA)
D = rel_mag_D*rel_rob_D

# Plot
p = bk.figure(title = "Robustness Relative to RGA",x_axis_label = "Sample No.", y_axis_label= "Relative Difference",height= 800, width = 800, y_range = [-1.2,1.2])
# Add the background color
p.add_layout(BoxAnnotation(top= 0, fill_alpha=0.1, fill_color='red'))
p.add_layout(BoxAnnotation(bottom= 0, fill_alpha=0.1, fill_color='green'))
p.scatter(rel_mag_D.index, D, color = TUBScolorscale[2], legend ="R2D2")
p.scatter( rel_mag_A.index, A, color = TUBScolorscale[0], legend ="Aström Decoupled")

# Show the plot
#bk.show(p)

# Show and Export
p.output_backend = "svg"
export_svgs(p, filename="MIMO_MS_RELROB_"+studyname+".svg")

# Clear Bokeh
bk.reset_output

<function bokeh.io.reset_output>

In [4]:
def calc_mean(samples):
    Mean = []
    X = []
    for n in range(1,9):
        Mean.append(np.mean(samples.iloc[(n-1)*1000:n*1000]))
        X.append(n)
    return X,Mean

sample_data = Data[Data['MS_RGA']>0.0]['MS_RGA']
samples = range(0,len(sample_data))
x_mean,sample_mean = calc_mean(sample_data)
p2 = bk.figure(title="Maximum Singular Values per Sample", x_axis_label = "Sample No.",
               y_axis_label="Maximum Singular Value",height= 800, width = 800, 
               y_axis_type = "log")
p3 = bk.figure(title="Mean Maximum Singular Values per Order", x_axis_label = "Process Order",
               y_axis_label="Mean Maximum Singular Value",height= 300, width = 800)
p2.scatter(samples,sample_data, color = TUBScolorscale[6], marker = "circle", legend = "RGA")
p3.line(x_mean, sample_mean, line_dash = "dashed", color =TUBScolorscale[6],legend = "RGA")
p3.scatter(x_mean, sample_mean,  color =TUBScolorscale[6], marker = "circle")
sample_data = Data[Data['MS_A']>0.0]['MS_A']
samples = range(0,len(sample_data))
x_mean,sample_mean = calc_mean(sample_data)
p2.scatter(samples,sample_data, color = TUBScolorscale[0], marker = "cross", legend = "Aström Decoupled")
p3.line(x_mean, sample_mean, line_dash = "dashed", color =TUBScolorscale[0],legend = "Aström Decoupled")
p3.scatter(x_mean, sample_mean, color =TUBScolorscale[0], marker = "cross")
sample_data = Data[Data['MS_D']>0.0]['MS_D']
samples = range(0,len(sample_data))
x_mean,sample_mean = calc_mean(sample_data)
p2.scatter(samples,sample_data, color = TUBScolorscale[2], marker = "diamond", legend = "Modified Aström")
p3.line(x_mean, sample_mean, line_dash = "dashed", color =TUBScolorscale[2],legend = "Modified", )
p3.scatter(x_mean, sample_mean,color =TUBScolorscale[2], marker = "diamond")

p3.legend.location = "top_left"
# Show
#bk.show(p2)
#bk.show(p3)

#Export
p2.output_backend = "svg"
export_svgs(p2, filename="MIMO_MS_OVER_SAMPLES_ALL_"+studyname+".svg")
p3.output_backend = "svg"
export_svgs(p3, filename="MIMO_MS_OVER_ORDER_"+studyname+".svg")

# Clear Bokeh
bk.reset_output

error: [Errno 10054] An existing connection was forcibly closed by the remote host

In [ ]:
# Initialize BIG Plot

# Scatter
p1 = bk.figure(x_axis_type = "log", x_axis_label ="Frequency of Maximum Singular Value", 
               y_axis_label = "Maximum Singular Value",
               height= 800, width = 800, 
               y_axis_type = "log", x_range = [1e-5,1e3])
# Add the good and bad 
p1.add_layout(BoxAnnotation(top = 1e3, right = 1e3,fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(top = 1.8, left = 1./180.,right=1e3, fill_alpha=0.1, fill_color='white'))
p1.add_layout(BoxAnnotation(top = 1.8, left = 1./180.,right=1e3, fill_alpha=0.1, fill_color='green'))
# Collect Data
sample_data = Data[(Data['MS_RGA']>0.0)]['MS_RGA']
samples = Data[Data['MS_RGA']>0.0]['w_MS_RGA']
# Add Scatter
p1.scatter(samples,sample_data, color = TUBScolorscale[6], marker = "circle", legend = "RGA")
# Collect Data
sample_data = Data[Data['MS_A']>0.0]['MS_A']
samples = Data[Data['MS_RGA']>0.0]['w_MS_A']
# Add Scatter
p1.scatter(samples,sample_data, color = TUBScolorscale[0],  marker = "cross", legend = "Aström Decoupled")
# Collect Data
sample_data = Data[Data['MS_D']>0.0]['MS_D']
samples = Data[Data['MS_RGA']>0.0]['w_MS_D']
# Add Scatter
p1.scatter(samples,sample_data, color = TUBScolorscale[2], marker = "diamond", legend = "Modified Aström")

#bk.show(p1)

# Show and Export
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_MS_OVER_Frequency_ALL_"+studyname+".svg")

# Clear Bokeh
bk.reset_output

In [ ]:
# Get the histogram of MS
from bokeh.layouts import gridplot, layout
from bokeh.models import BoxAnnotation, Label
from scipy.interpolate import UnivariateSpline

# Data range
MS_range = (0.1, 2)

sample_data = Data[['MS_RGA']]
# Histogram
hist, edges = np.histogram(sample_data, density = False, bins = 100, range=MS_range)

# Make a plot
p1 = bk.figure(title="Distribution of Maximum Singular Values", y_axis_label = "Samples",height= 300, width = 800)
# plot the areas
p1.add_layout(BoxAnnotation(left= 1.05, fill_alpha=0.1, fill_color='red'))
robust_text = Label(x=1.3, y=0.9*max(hist), text='Robustness Issues',text_font_style = "bold")
p1.add_layout(robust_text)
p1.add_layout(BoxAnnotation(left=0.8, right = 1.05, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=0.8, fill_alpha=0.1, fill_color='yellow'))
performance_text = Label(x=0.3, y=0.9*max(hist), text='Performance Issues',text_font_style = "bold")
p1.add_layout(performance_text)
# Add Data
p1.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[6])


sample_data = Data[['MS_A']]
hist, edges = np.histogram(sample_data, density = False, bins = 100, range=MS_range)
# Make plot
p2 = bk.figure(y_axis_label = "Samples",height= 300, width = 800)
# plot the areas
p2.add_layout(BoxAnnotation(left= 1.05, fill_alpha=0.1, fill_color='red'))
p2.add_layout(BoxAnnotation(left=0.8, right = 1.05, fill_alpha=0.1, fill_color='green'))
p2.add_layout(BoxAnnotation(right=0.8, fill_alpha=0.1, fill_color='yellow'))
# Add Data
p2.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[0])

sample_data = Data[['MS_D']]
hist, edges = np.histogram(sample_data, density = False, bins = 100, range=MS_range)
# Make plot
p3 = bk.figure(y_axis_label = "Samples", x_axis_label = "Maximum Singular Value",height= 300, width = 800)
# plot the areas
p3.add_layout(BoxAnnotation(left= 1.05, fill_alpha=0.1, fill_color='red'))
p3.add_layout(BoxAnnotation(left=0.8, right = 1.05, fill_alpha=0.1, fill_color='green'))
p3.add_layout(BoxAnnotation(right=0.8, fill_alpha=0.1, fill_color='yellow'))
# Add Data
p3.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[2])

# Show and Export
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_HIST_MS_RGA_"+studyname+".svg")

p2.output_backend = "svg"
export_svgs(p2, filename="MIMO_HIST_MS_A"+studyname+".svg")

p3.output_backend = "svg"
export_svgs(p3, filename="MIMO_HIST_MS_D"+studyname+".svg")

#p = layout([p1,p2,p3])
#bk.show(p)

# Clear Bokeh
bk.reset_output

In [ ]:
# Get the histogram of Frequency of MS

# Frequency Range
# DEPENDS ON THE STUDY!!!
# PTN -> T = 80...180 mean 130
# TSUM -> T = 180
freq_range = (1e-3, 1e1)
freq_eig = 1./180.

sample_data = Data[['w_MS_RGA']]
hist, edges = np.histogram(sample_data, density = False, bins =np.logspace(np.log10(freq_range[0]), np.log10(freq_range[1]), 100), range=freq_range)
# Make a plot
p1 = bk.figure(title="Distribution of Frequency corresponding to Maximum Singular Value", y_axis_label = "Samples",height= 300, width = 800, x_axis_type = "log")
# plot the areas
p1.add_layout(BoxAnnotation(right= 0.9*freq_eig, fill_alpha=0.15, fill_color='red'))
robust_text = Label(x= freq_range[0], y=0.9*max(hist), text='Robustness Issues',text_font_style = "bold")
p1.add_layout(robust_text)
p1.add_layout(BoxAnnotation(left=0.9*freq_eig, fill_alpha=0.15, fill_color='green'))
# Plot data
p1.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[6])# Get the robustness and performance limits




sample_data = Data[['w_MS_A']]
hist, edges = np.histogram(sample_data, density = False, bins =np.logspace(np.log10(freq_range[0]), np.log10(freq_range[1]), 100),range=freq_range)
# Make figure
p2 = bk.figure(y_axis_label = "Samples",height= 300, width = 800, x_axis_type = "log")
# plot the areas
p2.add_layout(BoxAnnotation(right= 0.9*freq_eig, fill_alpha=0.15, fill_color='red'))
p2.add_layout(BoxAnnotation(left=0.9*freq_eig, fill_alpha=0.15, fill_color='green'))
# Plot Data
p2.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[0])

sample_data = Data[['w_MS_D']]
hist, edges = np.histogram(sample_data, density = False,bins =np.logspace(np.log10(freq_range[0]), np.log10(freq_range[1]), 100),range=freq_range)
# Make figure
p3 = bk.figure(y_axis_label = "Samples", x_axis_label = "Frequency[rad/s]",height= 300, width = 800, x_axis_type = "log")
# plot the areas
p3.add_layout(BoxAnnotation(right= 0.9*freq_eig, fill_alpha=0.15, fill_color='red'))
p3.add_layout(BoxAnnotation(left=0.9*freq_eig, fill_alpha=0.15, fill_color='green'))
# Plot Data
p3.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[2])

# Show and Export
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_HIST_wMS_RGA_"+studyname+".svg")

p2.output_backend = "svg"
export_svgs(p2, filename="MIMO_HIST_wMS_A"+studyname+".svg")

p3.output_backend = "svg"
export_svgs(p3, filename="MIMO_HIST_wMS_D"+studyname+".svg")

# Show
#p = layout([p1,p2,p3])
#bk.show(p)

# Clear Bokeh
bk.reset_output